In [1]:
"""Visualization of results for the xTB geometry optimization with geomeTRIC"""

__author__    = "Roberto Di Remigio"
__credit__    = ["Roberto Di Remigio", "Xin Li"]

__copyright__ = "(c) 2021, ENCSS and PDC"
__license__   = "MIT"
__date__      = "2021-05-04"

<figure>
  <IMG SRC="../img/ENCCS-PDC-logos.jpg" WIDTH=150 ALIGN="right">
</figure>

# Visualizing the results of a geometry optimization

<div style="background: #efffed;
            border: 1px solid grey;
            margin: 8px 0 8px 0;
            text-align: center;
            padding: 8px; ">
    <i class="fa-play fa" 
       style="font-size: 40px;
              line-height: 40px;
              margin: 8px;
              color: #444;">
    </i>
    <div>
    To run the selected code cell, hit <pre style="background: #efffed">Shift + Enter</pre>
    </div>
</div>

The optimizer will output a file containing the geometries and the total energies of all conformations visited during the optimization. We will use [matplotlib](https://matplotlib.org/) to plot the energy as a function of optimization iteration and [py3Dmol](https://3dmol.csb.pitt.edu/index.html) to interactively visualize the conformations. We can animate this visualization using [ipywidgets](https://ipywidgets.readthedocs.io/en/7.6.3/).

The first thing to do is to upload your output file. We use an uploader Python widget for that purpose.

In [ ]:
import codecs
import re

import ipywidgets


class GeometryOptimizerUploader(ipywidgets.HBox):
    
    def __init__(self):
        super().__init__()
        self.geometries = []
        self.energies = []
        
        # define widgets
        uploader = ipywidgets.FileUpload(
            accept=".xyz",
            multiple=False
        )
        uploader.observe(self.on_upload_change, names='_counter')
        
        self.children = [uploader]

    def on_upload_change(self, change):
        if not change.new:
            return
        up = change.owner
        
        regex = re.compile(br"Iteration (?P<iteration>\d+) Energy (?P<energy>-\d+.\d+)", re.MULTILINE)
        for filename, data in up.value.items():
            print(f'uploaded {filename}')
            contents = data["content"]
            matches = regex.finditer(contents)
            self.energies = [float(m.group("energy")) for m in matches]
            # number of lines in each XYZ structure
            xyzs = codecs.decode(contents).splitlines()
            natoms = int(xyzs[0])
            lines_per_xyz = natoms + 2
            for lines in range(0, len(xyzs), lines_per_xyz):
                self.geometries.append("\n".join(xyzs[lines:lines+lines_per_xyz]))
        up.value.clear()
        up._counter = 0

up = GeometryOptimizerUploader()
up

The code above will, upon upload, extract the optimization trajectory for us. Executing the next cell will show the results of the optimization.

In [ ]:
import ipywidgets
import py3Dmol as p3d

%matplotlib widget
from matplotlib import pyplot as plt


# get list of geometries from uploader widget
geometries = up.geometries
# output widget with geometries
out_geometries = ipywidgets.Output()
out_geometries.clear_output(wait=True)

# display first geometry
with out_geometries:
    v = p3d.view(width=300, height=300)
    v.addModel(geometries[0], "xyz")
    v.setStyle({"stick": {}})
    v.zoomTo()
    v.show()
    
@out_geometries.capture(clear_output=True, wait=True)
def on_geometry_change(change):
    idx = change["new"]
    v = p3d.view(width=300, height=300)
    v.addModel(geometries[idx], "xyz")
    v.setStyle({"stick": {}})
    v.zoomTo()
    v.show()

# get list of energies from uploader widget
energies = up.energies
# output widget with energies
out_energies = ipywidgets.Output()
out_energies.clear_output(wait=True)

# display full trajectory plot with point for first geometry
with out_energies:
    fig, ax = plt.subplots(constrained_layout=True, figsize=(4, 2.5), num="Geometry optimization")
    line, = ax.plot(energies)
    ax.scatter(0, energies[0], s=10, c="red")

    # Labeling the axes
    ax.set_xlabel("Iteration")
    ax.set_ylabel("Energy (atomic units)")
    fig.canvas.toolbar_position = 'bottom'
    ax.grid(True)

@out_energies.capture(clear_output=True, wait=True)
def on_energy_change(change):
    idx = change["new"]
    ax.scatter(idx, energies[idx], s=10, c="red")
    fig.canvas.draw()
    
# a slider widgets, to select geometry to display
slider = ipywidgets.IntSlider(min=0, max=len(geometries)-1, step=1, continuous_update=True)
# a player widget, to show the whole optimization trajectory
player = ipywidgets.Play(min=0, max=len(geometries)-1, interval=400)
# put control widget in a vertical box widget
controls = ipywidgets.VBox([slider, player])

# link slider widget with geometry change
slider.observe(on_geometry_change, 'value')
# link slider widget with energy change
slider.observe(on_energy_change, 'value')
# link player and slider widgets
ipywidgets.jslink((player, 'value'), (slider, 'value'))
# put controls and output widget in horizontal box widget
ipywidgets.HBox([controls, out_geometries, out_energies])